In [3]:
gi.require_version('Gtk', '3.0')
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, Gtk
GObject.threads_init()
Gst.init(None)

NameError: name 'gi' is not defined